In [1]:
import numpy as np 
import healpy as hp
import matplotlib.pyplot as plt
from glob import glob
import mylib
from tqdm import tqdm
from importlib import reload
import pymaster as nmt

In [2]:
nside = 256
lmax = 3*nside - 1
npix = 12 * nside**2
nmc = 10

ell_input, TT, TE, EE, BB, PP = np.loadtxt('cl.txt', unpack=True)
c2d = ell_input*(ell_input+1)/2/np.pi
# input_cls = np.array([TT, EE, np.zeros_like(EE), TE]) #no BB
input_cls = np.array([TT, EE, BB, TE]) 
input_cls /= c2d 
for c in input_cls: c[0] = 0 
input_cls = input_cls[:,:lmax+1]

/tmp/ipykernel_43810/721740321.py:10: RuntimeWarning: invalid value encountered in divide
  input_cls /= c2d


In [ ]:
mapdir = 'noise_mc/out/'

coadded_noise = np.zeros((nmc, 3, npix))
hits = np.zeros((17, npix))

for mc in tqdm(range(nmc)):
    invcov = np.zeros((17, 6, npix))
    nw = np.zeros((17, 3, npix))
    for i, schedule in enumerate(sorted(glob(f'{mapdir}{mc:03d}/*'))):
        if mc == 0:
            hits[i] = hp.read_map(f'{schedule}/filterbin_hits.fits')
        # cov[i] = hp.read_map(f'{schedule}/filterbin_cov.fits', field=None)
        invcov[i] = hp.read_map(f'{schedule}/filterbin_invcov.fits', field=None)
        nw[i] = hp.read_map(f'{schedule}/filterbin_noiseweighted_filtered_map.fits', field=None)
        # m[i] = hp.read_map(f'{schedule}/filterbin_filtered_map.fits', field=None)
        # hits[i] = hp.read_map(f'{schedule}/filterbin_hits.fits')
        # rcond[i] = hp.read_map(f'{schedule}/filterbin_rcond.fits', field=None)
    if mc == 0:
        coadded_hits = np.sum(hits, axis=0)
    coadded_noise[mc] = mylib.coadd_IQU(invcov, nw)

 80%|███████████████████████████████████▏        | 8/10 [04:48<01:10, 35.38s/it]

In [ ]:
mapdir = 'signal_mc/out/'

coadded_signal = np.zeros((nmc, 3, npix))
hits = np.zeros((17, npix))

for mc in tqdm(range(nmc)):
    invcov = np.zeros((17, 6, npix))
    nw = np.zeros((17, 3, npix))
    for i, schedule in enumerate(sorted(glob(f'{mapdir}{mc:03d}/*'))):
        if mc == 0:
            hits[i] = hp.read_map(f'{schedule}/filterbin_hits.fits')
        # cov[i] = hp.read_map(f'{schedule}/filterbin_cov.fits', field=None)
        invcov[i] = hp.read_map(f'{schedule}/filterbin_invcov.fits', field=None)
        nw[i] = hp.read_map(f'{schedule}/filterbin_noiseweighted_filtered_map.fits', field=None)
        # m[i] = hp.read_map(f'{schedule}/filterbin_filtered_map.fits', field=None)
        # hits[i] = hp.read_map(f'{schedule}/filterbin_hits.fits')
        # rcond[i] = hp.read_map(f'{schedule}/filterbin_rcond.fits', field=None)
    if mc == 0:
        coadded_hits = np.sum(hits, axis=0)
    coadded_signal[mc] = mylib.coadd_IQU(invcov, nw)

In [ ]:
mapdir = 'full_mc/out/'

coadded_full = np.zeros((nmc, 3, npix))
hits = np.zeros((17, npix))

for mc in tqdm(range(nmc)):
    invcov = np.zeros((17, 6, npix))
    nw = np.zeros((17, 3, npix))
    for i, schedule in enumerate(sorted(glob(f'{mapdir}{mc:03d}/*'))):
        if mc == 0:
            hits[i] = hp.read_map(f'{schedule}/filterbin_hits.fits')
        # cov[i] = hp.read_map(f'{schedule}/filterbin_cov.fits', field=None)
        invcov[i] = hp.read_map(f'{schedule}/filterbin_invcov.fits', field=None)
        nw[i] = hp.read_map(f'{schedule}/filterbin_noiseweighted_filtered_map.fits', field=None)
        # m[i] = hp.read_map(f'{schedule}/filterbin_filtered_map.fits', field=None)
        # hits[i] = hp.read_map(f'{schedule}/filterbin_hits.fits')
        # rcond[i] = hp.read_map(f'{schedule}/filterbin_rcond.fits', field=None)
    if mc == 0:
        coadded_hits = np.sum(hits, axis=0)
    coadded_full[mc] = mylib.coadd_IQU(invcov, nw)

In [ ]:
mask = np.zeros_like(coadded_hits)
mask[coadded_hits!=0] = 1

In [ ]:
mask_apo = nmt.mask_apodization(mask, 6, apotype='C2')
hp.mollview(mask_apo)
fsky = len(mask_apo[(mask_apo!=0)])/npix
w2 = np.sum(mask_apo**2)/np.sum(mask)
w4 = np.sum(mask_apo**4)/np.sum(mask)

In [ ]:
b = nmt.NmtBin.from_nside_linear(nside, 20)
ells = b.get_effective_ells()
c2db = ells * (ells+1) /2/np.pi

In [ ]:
reload(mylib)
# fl_bb = mylib.get_fl(input_cls[2], cl_input, pcl_bb, bl, mll, fsky*w2)
fl_bb = mylib.get_fl(input_cls[2], pcl_bb, bl, mll)

In [ ]:
plt.plot(fl_bb)
plt.xlim([10, 600])
plt.ylim([0, 1])
plt.grid()
plt.xlabel('ell')

In [ ]:
nl = np.empty((nmc, 4, len(ells)))

for mc in tqdm(range(nmc)):    
    f = nmt.NmtField(mask_apo, [coadded_noise[mc][1], coadded_noise[mc][2]], beam=None, purify_b=False)
    nl[mc] = nmt.compute_full_master(f, f, b)

nl_mean = np.mean(nl, axis=0)

In [ ]:
cl = np.empty((nmc, 4, len(ells)))

for mc in tqdm(range(nmc)):
    f = nmt.NmtField(mask_apo, [coadded_full[mc][1], coadded_full[mc][2]], beam=None, purify_b=False)
    cl[mc] = nmt.compute_full_master(f, f, b, nl_mean)

In [15]:
cl_mean = c2db * np.mean(cl, axis=0)
cl_std = c2db * np.std(cl, axis=0)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].errorbar(ells, cl_mean[3], yerr=cl_std[3], fmt='.', label='BB')
axes[0].plot(ell_input, BB, label='input BB')
axes[0].set_title('BB')

axes[1].plot(ells, cl_std[3], marker='.',label='std BB')
# axes[1].plot(ells, c2db*(bpw@knox), label='knox')
axes[1].set_title('std(BB)')
# axes[1].set_ylim([1e-5,1e0])

for ax in axes.flatten():
    ax.legend()
    # ax.set_xlim([10,600])
    ax.loglog()
    ax.grid()
    ax.set_xlabel('ell')
    ax.set_ylabel('D_ell')